country — наименование страны;
date — дата наблюдения;
total_vaccinations — общее число введённых вакцин в стране на указанный день;
people_vaccinated — общее число привитых первым компонентом в стране на указанный день;
people_vaccinated_per_hundred — процент привитых первым компонентом в стране на указанный день (рассчитывается как );
people_fully_vaccinated — общее число привитых вторым компонентом в стране на указанный день (первый компонент уже был введён им ранее);
people_fully_vaccinated_per_hundred — процент привитых вторым компонентом в стране на указанный день (рассчитывается как );
daily_vaccination — ежедневная вакцинация (число вакцинированных в указанный день);
vaccines — комбинации вакцин, используемые в стране.

 date — дата наблюдения;
 province/state — наименование провинции/штата;
 country — наименование страны;
 confirmed — общее число зафиксированных случаев на указанный день;
 deaths — общее число зафиксированных смертей на указанный день;
 recovered — общее число выздоровлений на указанный день.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
%matplotlib inline
plt.style.use('default')

covid_url = 'https://raw.githubusercontent.com/Ar1sha/sf/master/Pandas/covid_data.csv'
vacc_url = 'https://raw.githubusercontent.com/Ar1sha/sf/master/Pandas/country_vaccinations.csv'
df = pd.read_csv(covid_url, sep=',')
vaccinations_data = pd.read_csv(vacc_url, sep=',')
vaccinations_data = vaccinations_data[
    ['country', 'date', 'total_vaccinations', 
     'people_vaccinated', 'people_vaccinated_per_hundred',
     'people_fully_vaccinated', 'people_fully_vaccinated_per_hundred',
     'daily_vaccinations', 'vaccines']
]
df = df.groupby(
    ['date', 'country'], 
    as_index=False
)[['confirmed', 'deaths', 'recovered']].sum()
df['active'] = df['confirmed'] - df['deaths'] - df['recovered']
df = df.sort_values(by=['country', 'date'])
df['daily_confirmed'] = df.groupby('country')['confirmed'].diff()
df['daily_deaths'] = df.groupby('country')['deaths'].diff()
df['daily_recovered'] = df.groupby('country')['recovered'].diff()
df['date'] = pd.to_datetime(df['date'])
vaccinations_data['date'] = pd.to_datetime(vaccinations_data['date'])
mask = df['date'].max()
mask2 = df['date'].min()
vac_mask = vaccinations_data[(vaccinations_data['date']<mask) & (vaccinations_data['date']>mask2)]
covid_df = pd.merge(
    df,
    vac_mask,
    on=['country','date'],
    how='left'
)
covid_df['death_rate']= covid_df['deaths']/covid_df['confirmed']*100
covid_df['recovered_rate']=covid_df['recovered']/covid_df['confirmed']*100


countries = ['Russia', 'Australia', 'Germany', 'Canada', 'United Kingdom']
croped_covid_df = covid_df[covid_df['country'].isin(countries)]

populations = pd.DataFrame([
    ['Canada', 37664517],
    ['Germany', 83721496],
    ['Russia', 145975300],
    ['Australia', 25726900],
    ['United Kingdom', 67802690]
    ],
    columns=['country', 'population']
)
croped_covid_df = croped_covid_df.merge(populations, on=['country'])
croped_covid_df['daily_confirmed_per_hundred'] = croped_covid_df['daily_confirmed'] / croped_covid_df['population'] * 100
# jointplot = sns.jointplot(
#     data=croped_covid_df, 
#     x='people_fully_vaccinated_per_hundred', 
#     y='daily_confirmed_per_hundred',
#     hue='country',
#     xlim = (0, 40),
#     ylim = (0, 0.1),
#     height=8,
# )
# pivot = croped_covid_df.pivot_table(
#     values='people_vaccinated_per_hundred',
#     columns='date',
#     index='country',
# )
# pivot.columns = pivot.columns.astype('string')
# heatmap = sns.heatmap(data=pivot, cmap='YlGnBu')
# heatmap.set_title('Тепловая карта вакцинации', fontsize=16)
#считаем средний процент выздоровлений для каждой страны

#считаем среднее ежедневно фиксируемое количество выздоровевших по странам
treemap_data = covid_df.groupby(
    by='country',
    as_index=False
)[['daily_recovered']].mean()


#преобразуем даты в строковый формат
choropleth_data = covid_df.sort_values(by='date')
choropleth_data['date'] = choropleth_data['date'].astype('string')

#строим график
countries=['United States', 'Russia', 'United Kingdom', 'Brazil', 'France']
scatter_data = covid_df[covid_df['country'].isin(countries)]

#строим график
fig = px.scatter_3d(
    data_frame=scatter_data, #DataFrame
    x = 'daily_confirmed', #ось абсцисс
    y = 'daily_deaths', #ось ординат
    z = 'daily_vaccinations', #ось аппликат
    color='country', #расцветка в зависимости от страны
    log_x=True, 
    log_y=True,
    width=1000,
    height=700
)

#отображаем график
fig.show()
fig.write_html("plotly/scatter_3d.html")



FileNotFoundError: [Errno 2] No such file or directory: 'plotly\\scatter_3d.html'